# Lista 6

In [14]:
#| echo: False
import numpy as np
import pandas as pd
from scipy.stats import chi2

In [39]:
# Dane dla sprawdzenia poprawnosci
full_A = np.array([28, 89, 175, 195, 309, 462])
censored_A = np.array([377, 393, 421, 447, 709, 744, 770, 1106, 1206])
data_A = np.concatenate([full_A, censored_A])
delta_A = np.array([True]*len(full_A) + [False]*len(censored_A))
 
full_B = np.array([34, 88, 137, 199, 280, 291, 309, 351, 358, 369,
                   369, 370, 375, 382, 392, 451])
censored_B = np.array([299, 300, 429, 1119])
data_B = np.concatenate([full_B, censored_B])
delta_B = np.array([True]*len(full_B) + [False]*len(censored_B))

df = np.concatenate([data_A, data_B])
df_uc = np.unique(np.concatenate([full_A, full_B]))
df_uc = np.sort(df_uc)
d = pd.DataFrame({
    'Data': np.concatenate([data_A, data_B]),
    'Delta': np.concatenate([delta_A, delta_B]),
    'Groups': np.concatenate([np.zeros(len(data_A)), np.ones(len(df) - len(data_A))])
})

In [40]:
def tests(test, df):
    t = np.asarray(df["Data"])
    delta = np.asarray(df["Delta"])
    group = np.asarray(df["Groups"])
    
    groups = np.unique(group)
    k = len(groups)
    
    delta_t = np.sort(np.unique(t[delta == 1]))
    
    Z = np.zeros(k - 1)            
    Sigma = np.zeros((k - 1, k - 1))
    
    for t_i in delta_t:
        at_risk = t >= t_i
        r_i = np.sum(at_risk)
        d_i = np.sum((t == t_i) & (delta == 1))
        
        r_ij = np.array([np.sum(at_risk & (group == g)) for g in groups])
        d_ij = np.array([np.sum((t == t_i) & (delta == 1) & (group == g)) for g in groups])
        
        if test == "log":
            W = 1.0
        elif test == "gehan":
            W = r_i
        elif test == "tarone":
            W = np.sqrt(r_i)
        elif test == "peto":
            W = 1.0
            for t_j in delta_t:
                if t_j > t_i:
                    break
                r_j = np.sum(t >= t_j)
                d_j = np.sum((t == t_j) & (delta == 1))
                W *= (1 - d_j / (r_j+1))
        
        for j in range(k - 1):
            Z[j] += W * (d_ij[j] - r_ij[j] * (d_i / r_i))
        
        for i in range(k - 1):
            for j in range(k - 1):
                Sigma[i, j] += W**2 * d_i * (r_ij[i]/r_i) * ((int(i==j) - r_ij[j]/r_i) * ((r_i - d_i)/(r_i - 1)))
    
    Z2 = Z @ np.linalg.inv(Sigma) @ Z.T
    p_value = 1 - chi2.cdf(Z2, df=k-1)
    
    return p_value

In [41]:
print(tests("log", d))
print(tests("gehan", d))
print(tests("tarone", d))
print(tests("peto", d))

0.018308394191641653
0.1342335353090518
0.055004544098656516
0.1014675578613431
